# Extracting parameters

In [1]:
import numpy as np
import time
import qutip as qtp
import pysqkit
from pysqkit import QubitSystem, Qubit
from pysqkit.util.linalg import get_mat_elem
from pysqkit.util.phys import temperature_to_thermalenergy
from typing import List, Dict, Callable
import json
import cmath

from IPython.display import display, Latex

The following file contains the parameters used in the simulations. Parameter set 2 and 3 are the relevant ones for the Cross-Resonance gate

In [2]:
with open('flx_transm_params.txt') as param_file:
    parameters_set = json.load(param_file)

In [3]:
temperature = 0.020 # K
thermal_energy = temperature_to_thermalenergy(temperature) # kb T/h in GHz
d_comp = 4

p_set = "2"


#Transmon
levels_t = 3
transm = pysqkit.qubits.SimpleTransmon(
    label='T', 
    max_freq=parameters_set[p_set]["max_freq_t"], 
    anharm=parameters_set[p_set]["anharm_t"],
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_t"], 
    env_thermal_energy=thermal_energy,    
    dim_hilbert=levels_t,
    dephasing_times=parameters_set[p_set]["dephasing_times_t"]
)

#Fluxonium
levels_f = 4

flx = pysqkit.qubits.Fluxonium(
    label='F', 
    charge_energy=parameters_set[p_set]["charge_energy_f"], 
    induct_energy=parameters_set[p_set]["induct_energy_f"], 
    joseph_energy=parameters_set[p_set]["joseph_energy_f"],  
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_f"], 
    env_thermal_energy=thermal_energy,
    dephasing_times= parameters_set[p_set]["dephasing_times_f"] #ns/2*np.pi 
)
flx.diagonalize_basis(levels_f)

# We also add a drive on the fluxonium
flx.add_drive(
    pysqkit.drives.microwave_drive,
    label='cz_drive_f',
    pulse=pysqkit.drives.pulses.cos_modulation,
    pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
)

d_leak = levels_t*levels_f - d_comp

jc = parameters_set[p_set]["jc"]
coupled_sys = transm.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)
bare_system = transm.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=0.0)

freq_flx = flx.state('1')[0] - flx.state('0')[0]
freq_03_flx = flx.state('3')[0] - flx.state('0')[0]
#freq_34_flx = flx.state('4')[0] - flx.state('3')[0]

state_label = ["00", "01", "10", "11"]
comp_states = {}
for label in state_label:
    state_tmp = coupled_sys.state(label)[1]
    loc = np.argmax(np.abs(state_tmp))
    phase = cmath.phase(state_tmp[loc])
    state_tmp = np.exp(-1j*phase)*state_tmp
    comp_states[label] = state_tmp

eps_drive = 0.6 #parameter taken in simulations

In [4]:
def mu_yz_flx(
    comp_states: Dict, 
    system: QubitSystem,
    eps_drive: float
) -> float:
    
    """
    Description
    --------------------------------------------------------------
    Returns the YZ coefficient associated with the charge 
    operator of the fluxonium in the dressed basis.
    """
    
    op = coupled_sys["F"].charge_op()
    yz0 = get_mat_elem(op, comp_states['00'], comp_states['10'])
    yz1 = get_mat_elem(op, comp_states['01'], comp_states['11'] )
    return np.abs((np.imag(yz0 - yz1))/2)*eps_drive/2

def gate_time_cr(
    comp_states: Dict, 
    system: QubitSystem, 
    eps_drive: float
) -> float:
    
    """
    Description
    --------------------------------------------------------------
    Returns the gate time for an ideal cross-resonance gate with
    constant envelope.
    """
    
    coeff = mu_yz_flx(comp_states, system, eps_drive)
    return 1/(8*coeff)  
    

def zz(
    system: QubitSystem
) -> float:
    
    """
    Description
    --------------------------------------------------------------
    Returns the residual ZZ coupling at zero drive.
    """
    
    xi_zz = system.state('00')[0] + system.state('11')[0] \
        - system.state('01')[0] - system.state('10')[0]
    return xi_zz

def delta(
    system: QubitSystem
) -> float:
    
    """
    Description
    --------------------------------------------------------------
    Returns the parameter Delta that determines the gate speed 
    of the CPHASE gate.
    """
        
    delta_gate = (system.state('13')[0] - system.state('10')[0]) - \
        (system.state('03')[0] - system.state('00')[0])
    return delta_gate 

The following is a list of the relevant parameters

In [5]:
display(Latex(r'$\omega_{{01, flx}}/2 \pi = {:.3f} \, \mathrm{{GHz}}$'.format(freq_flx)))
display(Latex(r'$\omega_{{01, transm}}/2 \pi = {:.3f} \, \mathrm{{GHz}}$'.format(transm.max_freq)))
#display(Latex(r'$\omega_{{34, flx}}/2 \pi = {:.3f} \, \mathrm{{GHz}}$'.format(freq_34_flx)))
display(Latex(r'$\omega_{{03, flx}}/2 \pi = {:.3f} \, \mathrm{{GHz}}$'.format(freq_03_flx)))
display(Latex(r'$\xi_{{ZZ}}/h = {:.3f} \, \mathrm{{MHz}}$'.format(1e3*zz(coupled_sys))))
display(Latex(r'$t_{{gate}}^{{(CR)}} = {:.3f} \, \mathrm{{ns}}$'.format(gate_time_cr(comp_states, coupled_sys, eps_drive))))
display(Latex(r'$\Delta/h = {:.3f} \, \mathrm{{MHz}}$'.format(1e3*delta(coupled_sys))))
display(Latex(r'$T_{{1,transm}} = {:.3f} \, \mu s$'.format(1e-3*1/(2*np.pi*transm.loss_rates(0, 1)[0]))))
display(Latex(r'$T_{{1,flx}}^{{1 \mapsto 0}} = {:.3f} \, \mu s$'.format(1e-3*1/(2*np.pi*flx.loss_rates(0, 1)[0]))))
display(Latex(r'$T_{{1,flx}}^{{0 \mapsto 1}} = {:.3f} \, \mu s$'.format(1e-3*1/(2*np.pi*flx.loss_rates(0, 1)[1]))))
display(Latex(r'$T_{{1,flx}}^{{3 \mapsto 0}} = {:.3f} \, \mu s$'.format(1e-3*1/(2*np.pi*flx.loss_rates(0, 3)[0]))))
display(Latex(r'$T_{{1,flx}}^{{0 \mapsto 3}} = {:.3f} \, \mu s$'.format(1e-3*1/(2*np.pi*flx.loss_rates(0, 3)[1]))))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

The following function performs the numerical SW between coupled transmon and fluxonium in the presence of drive

In [9]:
def dressed_state(
    lev: str,
    hamil: qtp.qobj.Qobj(),
    lev_t: int,
    lev_f: int
):
    """
    Description
    ---------------------------------------------------------
    Function that returns the dressed state
    """
    
    #eig_en_0, eig_states_0 = h_0.eigenstates()
    eig_en, eig_states = hamil.eigenstates()
    
    lev_list = [int(index) for index in lev]
    
    if lev_list[0] >= lev_t or lev_list[1] >= lev_f:
        raise ValueError("Index error: indeces out of bound")
    state_0 = qtp.tensor(qtp.basis(lev_t, lev_list[0]), qtp.basis(lev_f, lev_list[1]))
    #en_0 = eig_en_0[state_index]
    eig_states_list = list(eig_states)
    state = eig_states_list[0]
    en = eig_en[0]
    fid = qtp.fidelity(state*state.dag(), state_0*state_0.dag())
    for state_tmp, en_tmp in zip(eig_states_list, eig_en):
        fid_tmp = qtp.fidelity(state_tmp*state_tmp.dag(), state_0*state_0.dag())
        if fid_tmp > fid:
            state = state_tmp
            fid = fid_tmp
            en = en_tmp
    loc = np.argmax(np.abs(np.array(state)) )
    phase = cmath.phase(state[loc])
    state = np.exp(-1j*phase)*state
    return en, state

def projector(
    hamil: qtp.qobj.Qobj(),
    lev_t: int,
    lev_f: int,
    labels=["00", "01", "10", "11"]
):
    states = {}
    proj = 0
    for label in labels:
        states[label] = dressed_state(label, hamil, lev_t, lev_f)[1]
        proj += states[label]*states[label].dag()
    return proj, states

def h_sw_exact(
    proj_0: qtp.qobj.Qobj(),
    proj: qtp.qobj.Qobj(),
    hamil: qtp.qobj.Qobj(),
    states_0: dict,
    lev_t: int,
    lev_f: int
):
    refl_0 = qtp.tensor(qtp.qeye(lev_t), qtp.qeye(lev_f)) - 2*proj_0
    refl = qtp.tensor(qtp.qeye(lev_t), qtp.qeye(lev_f)) - 2*proj
    u_sw = (refl_0*refl).sqrtm()
    h_sw = u_sw*hamil*u_sw.dag()
    h_eff = np.zeros([4, 4], dtype=complex)
    for key_row, row in zip(states_0.keys(), range(0, 4)):
        for key_col, col in zip(states_0.keys(), range(0, 4)):
            h_eff[row, col] = hamil.matrix_element(states_0[key_row], states_0[key_col])
    return h_eff

def get_pauli_coeff(
    op: np.ndarray,
    label: str
):
    pauli = {}
    pauli["I"] = np.identity(2, dtype=complex)
    pauli["X"] = np.array([[0.0, 1.0], [1.0, 0.0]], dtype=complex)
    pauli["Y"] = np.array([[0.0, -1j*1.0], [1j*1.0, 0.0]], dtype=complex)
    pauli["Z"] = np.array([[1.0, 0.0], [0.0, -1.0]], dtype=complex)
    label_list = [x for x in label]
    pauli_op = np.kron(pauli[label_list[0]], pauli[label_list[1]])
    coeff = 1/4*np.trace(op.conj().T.dot(pauli_op))
    return coeff

def get_params_sw(
    transm: Qubit,
    flx: Qubit,
    jc: float,
    drive_params: Dict
):
    """
    Description
    -----------------------------------------------------------------------
    Returns the exact cross-resonance coefficient in MHz.
    """
    
    if flx.dim_hilbert != 4:
        raise ValueError("Fluxonium dimension error: 4 levels of the fluxonium"
                        " must be included")
    if transm.dim_hilbert != 3:
        raise ValueError("Transmon dimension error: 3 levels of the fluxonium"
                        " must be included")
    
    freq_drive = drive_params["freq"]
    eps_drive = drive_params["eps"]
    phase_drive = np.pi/2
    
    eig_flx = flx.eig_energies()
    
    lev_t = 3
    lev_f = 4
    id_t = qtp.qeye(lev_t)
    id_f = qtp.qeye(lev_f)
    proj_1_t = qtp.basis(lev_t, 1)*qtp.basis(lev_t, 1).dag()
    proj_2_t = qtp.basis(lev_t, 2)*qtp.basis(lev_t, 2).dag()
    h_t_single = (transm.freq - freq_drive)*proj_1_t + (transm.anharm + 2*(transm.freq - freq_drive))*proj_2_t
    h_t = qtp.tensor(h_t_single, id_f)

    # Fluxonium Hamiltonian
    proj_1_f = qtp.basis(lev_f, 1)*qtp.basis(lev_f, 1).dag()
    proj_2_f = qtp.basis(lev_f, 2)*qtp.basis(lev_f, 2).dag()
    proj_3_f = qtp.basis(lev_f, 3)*qtp.basis(lev_f, 3).dag()
    ground_en_f = flx.eig_energies()[0]
    h_f_single = flx.hamiltonian(as_qobj=True) - ground_en_f*id_f - freq_drive*proj_1_f - \
        freq_drive*proj_3_f - 2*freq_drive*proj_2_f
    h_f = qtp.tensor(id_t, h_f_single)

    # Coupling Hamiltonian
    coup_op_t = 1j*transm.charge_zpf*(qtp.basis(lev_t, 1)*qtp.basis(lev_t, 0).dag() + np.sqrt(2)*qtp.basis(lev_t, 2)*qtp.basis(lev_t, 1).dag())
    q_10_f = flx.charge_op(as_qobj=True)[1, 0]
    q_30_f = flx.charge_op(as_qobj=True)[3, 0]
    q_21_f = flx.charge_op(as_qobj=True)[2, 1]
    coup_op_f = q_10_f*qtp.basis(lev_f, 1)*qtp.basis(lev_f, 0).dag() + \
        q_30_f*qtp.basis(lev_f, 3)*qtp.basis(lev_f, 0).dag() + q_21_f*qtp.basis(lev_f, 2)*qtp.basis(lev_f, 1).dag()
    h_coup = jc*(qtp.tensor(coup_op_t, coup_op_f.dag()) + qtp.tensor(coup_op_t, coup_op_f.dag()).dag())

    # Drive Hamiltonian
    h_drive = eps_drive/2*(qtp.tensor(id_t, coup_op_f).dag()*np.exp(1j*phase_drive) + \
                           qtp.tensor(id_t, coup_op_f)*np.exp(-1j*phase_drive))

    #Total Hamiltonian
    h_0 = h_t + h_f
    h = h_0 + h_coup
    h_tot = h + h_drive
    
    proj_dressed, states_dressed = projector(h, lev_t, lev_f)
    proj_dressed_drive, states_dressed_drive = projector(h_tot, lev_t, lev_f)
    ref_dressed = qtp.tensor(id_t, id_f) - 2*proj_dressed
    ref_dressed_drive = qtp.tensor(id_t, id_f) - 2*proj_dressed_drive
    u_sw = (ref_dressed*ref_dressed_drive).sqrtm()
    
    h_sw_eff = h_sw_exact(proj_dressed, proj_dressed_drive, h_tot, states_dressed, lev_t, lev_f)
    
    output = {}
    output["transm_freq"] = transm.freq # GHz
    output["flx_freq_01"] = eig_flx[1] - eig_flx[0] #GHz
    output["flx_freq_12"] = eig_flx[2] - eig_flx[1] #GHz
    output["flx_freq_03"] = eig_flx[3] - eig_flx[0] #GHz
    output["eps"] = eps_drive #GHz
    output["cr"] = np.real(get_pauli_coeff(h_sw_eff, "XZ")*1e3) #MHz
    output["cr_gate_time"] = 1/(8*np.abs(output["cr"])/1e3) #ns
    output["zz"] = 4*np.real(get_pauli_coeff(h_sw_eff, "ZZ")*1e3) #Mhz (notice factor of 4)
    output["jc"] = jc*1e3 #MHz
    
    return output

In [10]:
levels_f = 4

flx = pysqkit.qubits.Fluxonium(
    label='F', 
    charge_energy=parameters_set[p_set]["charge_energy_f"], 
    induct_energy=parameters_set[p_set]["induct_energy_f"], 
    joseph_energy=parameters_set[p_set]["joseph_energy_f"], #8.0, 
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_f"], #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,
    dephasing_times= parameters_set[p_set]["dephasing_times_f"] #ns/2*np.pi 
)
flx.diagonalize_basis(levels_f)

result_sw = get_params_sw(transm, flx, jc, {"freq": transm.freq, "eps":0.6})

In [11]:
result_sw

{'transm_freq': 4.5,
 'flx_freq_01': 0.5818489963974303,
 'flx_freq_12': 3.3885865584520825,
 'flx_freq_03': 6.574488472163283,
 'eps': 0.6,
 'cr': -1.0695142170538472,
 'cr_gate_time': 116.87549170158117,
 'zz': 0.14220174015155962,
 'jc': 20.0}

# Some remarks

One can notice that the ZZ coupling does not depend at all on the drive strength. The small discrepancies (smaller than 10%) between the SW result and the full numerics on the ZZ should be due to the neglected fast rotating terms.